In [1]:
import logging

import numpy

import cicada.additive
from cicada.communicator import SocketCommunicator

logging.basicConfig(level=logging.INFO)

def main(communicator):
    log = cicada.Logger(logging.getLogger(), communicator)
    protocol = cicada.additive.AdditiveProtocol(communicator)

    # Player 0 will provide a secret dividend.
    a = numpy.array(37) if communicator.rank == 0 else None
    # All players know the divisor.
    b = numpy.array(7)
    
    log.info(f"Dividend: {a}", src=0)
    log.info(f"Divisor: {b}", src=1)

    # Compute the quotient
    a_share = protocol.share(src=0, secret=protocol.encoder.encode(a), shape=())
    quotient_share = protocol.untruncated_private_public_divide(a_share, b)
    quotient_share = protocol.truncate(quotient_share)
    quotient = protocol.encoder.decode(protocol.reveal(quotient_share))
    log.info(f"Player {communicator.rank} quotient: {quotient}")

SocketCommunicator.run(world_size=2, fn=main);

INFO:root:Dividend: 37
INFO:root:Divisor: 7
INFO:root:Player 0 quotient: 5.285552978515625
INFO:root:Player 1 quotient: 5.285552978515625


As you can see, the result revealed to both players is a close approximation to the expected value.